## LLM-Blender Usage examples

Please first download our DeBERTa-v3-large PairRanker checkpoint to your local folder: [checkpoint link](https://drive.google.com/drive/folders/1E3qsZqja5IBaYEDRtVARU88mDl_nBqQ3?usp=sharing).
And put that to `<your checkpoint path>`

In [1]:
import llm_blender
ranker_config = llm_blender.RankerConfig
ranker_config.ranker_type = "pairranker"
ranker_config.model_type = "deberta"
ranker_config.model_name = "microsoft/deberta-v3-large" # ranker backbone
ranker_config.load_checkpoint = "checkpoint-best" # ranker checkpoint <your checkpoint path>
ranker_config.cache_dir = "./hf_models" # hugging face model cache dir
ranker_config.source_max_length = 128
ranker_config.candidate_max_length = 128
ranker_config.n_tasks = 1 # number of singal that has been used to train the ranker. This checkpoint is trained using BARTScore only, thus being 1.
fuser_config = llm_blender.GenFuserConfig
fuser_config.model_name = "llm-blender/gen_fuser_3b" # our pre-trained fuser
fuser_config.cache_dir = "./hf_models"
fuser_config.max_length = 512
fuser_config.candidate_max_length = 128
blender_config = llm_blender.BlenderConfig
blender_config.device = "cuda" # blender ranker and fuser device
blender = llm_blender.Blender(blender_config, ranker_config, fuser_config)

/home/dongfu/.conda/envs/llm_reranker/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-06-04 23:00:10.305583: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/home/dongfu/.conda/envs/llm_reranker/lib/python3.9/site-packages/transformers/convert_slow_tokenizer.py:454: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the orig


Using DeBERTa model


Some weights of the model checkpoint at microsoft/deberta-v3-large were not used when initializing DebertaV2Model: ['lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.dense.bias', 'mask_predictions.classifier.weight', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.dense.weight', 'lm_predictions.lm_head.dense.weight', 'mask_predictions.classifier.bias', 'mask_predictions.LayerNorm.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.bias']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## Using LLM-Blender for ranking
By the rank function, LLM-Blender could ranks the candidates through pairwise comparisons and return the ranks. We show our ranker's ranks are highly correlated with ChatGPT ranks.

In [2]:
import datasets
from llm_blender.gpt_eval.cor_eval import COR_MAPS
from llm_blender.gpt_eval.utils import get_ranks_from_chatgpt_cmps
mixinstruct_test = datasets.load_dataset("llm-blender/mix-instruct", split="test", streaming=True)
few_examples = list(mixinstruct_test.take(8))
# remove cmp_results with none cmp results
for ex in few_examples:
    keys = list(ex['cmp_results'].keys())
    for key in keys:
        if not ex['cmp_results'][key]:
            del ex['cmp_results'][key]
few_examples = [x for x in few_examples if x['cmp_results']]
inputs = [x['input'] for x in few_examples]
candidates_texts = [[cand['text'] for cand in x['candidates']] for x in few_examples]
print("Example:")
print("input:", inputs[0])
print("candidates:", candidates_texts[0])

Example:
input: I've always wondered what the difference is between a skeptic and a denier.
candidates: ['A skeptic is someone who doubts or expresses doubt about a claim or idea without being dismissive of it. They are open-minded and approach evidence with an open mind, searching for reasonable explanations and evidence to support their beliefs.\n\nA denier, on the other hand, is someone who actively works to deny or ignore evidence that contradicts their beliefs. They are often characterized by a closed mind and an unwillingness to consider alternative perspectives. They may also use rhetoric or false claims to try to discredit the evidence.', "Can you explain?\n5.   I've also noticed that some people who are skeptical about climate change also tend to be skeptical about other scientific subjects, like evolution. Can you explain that?\n6.   What evidence have you seen that supports the theory of evolution?\n\nThese are just a few examples of questions that a journalist might ask to 

In [3]:

ranks = blender.rank(inputs, candidates_texts, return_scores=False, batch_size=2)
print("ranks:", ranks)

Ranking candidates: 100%|██████████| 4/4 [00:44<00:00, 11.18s/it]

ranks: [[ 1 11  4  9 12  5  2  8  6  3 10  7]
 [ 1  9  3  2 12 10  8 11  4  6  5  7]
 [12  9  5  6  7 11  3  4  1  2 10  8]
 [ 2  7  4  3 12 10  8  9  1  5 11  6]
 [ 2  1  3  5 12  8 11  7  6  4  9 10]
 [ 2 11  3  5 12 10  1  7  4  6  9  8]
 [ 4  3  9  7 12  1 10  6  8  5 11  2]
 [ 2  9  7  3 12 10  1 11  4  5  8  6]]


In [4]:
import numpy as np
llm_ranks_map, gpt_cmp_results = get_ranks_from_chatgpt_cmps(few_examples)
gpt_ranks = np.array(list(llm_ranks_map.values())).T
print("Correlation with ChatGPT")
print("------------------------")
for cor_name, cor_func in COR_MAPS.items():
    print(cor_name, cor_func(ranks, gpt_ranks))

Correlation with ChatGPT
------------------------
pearson 0.502434644007648
spearman 0.35554809046205055
spearman_footrule 25.5
set_based 0.6422190656565656


## Using LLM-blender to directly compare two candidates

In [5]:
candidates_A = [x['candidates'][0]['text'] for x in few_examples]
candidates_B = [x['candidates'][1]['text'] for x in few_examples]
comparison_results = blender.compare(inputs, candidates_A, candidates_B, batch_size=2)
print("comparison_results:", comparison_results)

Ranking candidates: 100%|██████████| 4/4 [00:00<00:00,  5.70it/s]

comparison_results: [ True  True False  True False  True  True  True]


## Using LLM-Blender for fuse generation
We show that the the fused generation using the top-ranked candidate from the rankers could get outputs of higher quality.

In [6]:
from llm_blender.blender.blender_utils import get_topk_candidates_from_ranks
topk_candidates = get_topk_candidates_from_ranks(ranks, candidates_texts, top_k=3)
fuse_generations = blender.fuse(inputs, topk_candidates, batch_size=2)
print("fuse_generations:", fuse_generations)

Fusing candidates: 100%|██████████| 4/4 [00:14<00:00,  3.64s/it]

fuse_generations: ["A skeptic is someone who questions the validity of a claim or idea, while a denier is someone who dismisses or ignores evidence that contradicts their beliefs. Skeptics approach claims with an open mind and seek evidence to support or refute them, while denier's often have a closed mind and refuse to consider evidence that contradicts their beliefs.", "One number word that has the same number of letters as the number is 'one'.", 'Here is an example of a count from 1 to 5 in Python:  python i = 1; while (i = 5)  console.writeLine(i); i++;   This will output:  1 2 3 4 5 ', 'Air pollution can be monitored using a variety of technologies such as air quality monitors, remote sensing, satellites, and ground-based sensors. Some common types of air quality monitors include portable air quality monitors, stationary air quality monitors, and mobile air quality monitors.', 'Peanut butter is generally considered safe for most people to eat in moderation. However, if you have a 

In [7]:
# # Or do rank and fuser together
fuse_generations, ranks = blender.rank_and_fuse(inputs, candidates_texts, return_scores=False, batch_size=2, top_k=3)

Fusing candidates: 100%|██████████| 4/4 [00:14<00:00,  3.74s/it]


In [8]:
from llm_blender.common.evaluation import overall_eval
metrics = ['bartscore']
targets = [x['output'] for x in few_examples]
scores = overall_eval(fuse_generations, targets, metrics)

print("Fusion Scores")
for key, value in scores.items():
    print("  ", key+":", np.mean(value))

print("LLM Scores")
llms = [x['model'] for x in few_examples[0]['candidates']]
llm_scores_map = {llm: {metric: [] for metric in metrics} for llm in llms}
for ex in few_examples:
    for cand in ex['candidates']:
        for metric in metrics:
            llm_scores_map[cand['model']][metric].append(cand['scores'][metric])
for i, (llm, scores_map) in enumerate(llm_scores_map.items()):
    print(f"{i} {llm}")
    for metric, llm_scores in llm_scores_map[llm].items():
        print("  ", metric+":", np.mean(llm_scores))


Evaluating bartscore: 100%|██████████| 8/8 [00:00<00:00, 39.97it/s]

Fusion Scores
   bartscore: -3.8043667674064636
LLM Scores
0 oasst-sft-4-pythia-12b-epoch-3.5
   bartscore: -3.807092547416687
1 koala-7B-HF
   bartscore: -4.550534904003143
2 alpaca-native
   bartscore: -4.206288725137711
3 llama-7b-hf-baize-lora-bf16
   bartscore: -3.9363586008548737
4 flan-t5-xxl
   bartscore: -4.934148460626602
5 stablelm-tuned-alpha-7b
   bartscore: -4.432858616113663
6 vicuna-13b-1.1
   bartscore: -4.20223930478096
7 dolly-v2-12b
   bartscore: -4.440025061368942
8 moss-moon-003-sft
   bartscore: -3.587637573480606
9 chatglm-6b
   bartscore: -3.7075400948524475
10 mpt-7b
   bartscore: -4.1352817714214325
11 mpt-7b-instruct
   bartscore: -4.282741814851761
